In [ ]:
# numpy downgrade가 필요할 수 있음
#!python -m pip install numpy==1.2

In [1]:
#!pip install ensemble_boxes

     |████████████████████████████████| 3.3 MB 37.8 MB/s eta 0:00:01
     |████████████████████████████████| 26.3 MB 88.5 MB/s eta 0:00:01


In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO
from sklearn.preprocessing import normalize

In [50]:
# ensemble csv files
submission_files = ['/opt/ml/detection/mmdetection_new/boostcamp/submissions/swinB_cascade_rcnn/subm_swinB_fold1_1009.csv',
                    '/opt/ml/detection/mmdetection_new/boostcamp/submissions/swinB_cascade_rcnn/subm_swinB_fold2_1009.csv',
                    '/opt/ml/detection/mmdetection_new/boostcamp/submissions/swinB_cascade_rcnn/subm_swinB_fold3_1009.csv',
                    '/opt/ml/detection/mmdetection_new/boostcamp/submissions/swinB_cascade_rcnn/subm_swinB_fold4_1009.csv',
                    '/opt/ml/detection/mmdetection_new/boostcamp/submissions/swinB_cascade_rcnn/subm_swinB_fold5_1009.csv',
                    
                    '/opt/ml/detection/mmdetection_new/boostcamp/submissions/universe101/universe101_kfold1.csv',
                    '/opt/ml/detection/mmdetection_new/boostcamp/submissions/universe101/universe101_kfold2.csv',
                    '/opt/ml/detection/mmdetection_new/boostcamp/submissions/universe101/universe101_kfold3.csv',
                    '/opt/ml/detection/mmdetection_new/boostcamp/submissions/universe101/universe101_kfold4.csv',
                    '/opt/ml/detection/mmdetection_new/boostcamp/submissions/universe101/universe101_kfold5.csv',
                    
                    '/opt/ml/detection/mmdetection_new/boostcamp/submissions/yolor/yolor_LB6110.csv',
                    
                    '/opt/ml/detection/mmdetection_new/boostcamp/submissions/jsw_swinB_tta.csv',

                    '/opt/ml/detection/mmdetection_new/boostcamp/work_dirs/multi_scale_HTC_swinB_pafpn_full_batch/multi_scale_HTC_swinB_pafpn_full_batch_max300Img_submission_.csv',

                    ] # submission lists

submission_df = [pd.read_csv(file) for file in submission_files]

image_ids = submission_df[0]['image_id'].tolist()

annotation = '/opt/ml/detection/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [43]:
lb_scores = np.array([0.700]*5 + [0.618]*5 + [0.618] + [0.657]) #+ [0.647])
# weights = normalize(lb_scores[:,np.newaxis], axis=0).ravel()
# weights
weights = lb_scores/lb_scores.max()

In [5]:
np.exp(lb_scores*5)/np.exp(lb_scores*5).sum()

array([0.10216129, 0.10216129, 0.10216129, 0.10216129, 0.10216129,
       0.06779937, 0.06779937, 0.06779937, 0.06779937, 0.06779937,
       0.06779937, 0.08239732])

In [51]:
prediction_strings = []
file_names = []
iou_thr = 0.55

ensemble_function = weighted_boxes_fusion

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df_idx,df in enumerate(submission_df):
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        #scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        ### weight version
        score_list = np.array(list(map(float, predict_list[:, 1].tolist()))) # * weights[df_idx]
        scores_list.append(score_list.tolist()) 
        ###
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))

    if len(boxes_list):
        boxes, scores, labels = ensemble_function(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            if len(prediction_string) < 32000: ## excel 저장 수 제한
                score = round(score,6)
                #prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
                prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(round(box[0] * image_info['width'],3)) + ' ' + str(round(box[1] * image_info['height'],3)) + ' ' + str(round(box[2] * image_info['width'],3)) + ' ' + str(round(box[3] * image_info['height'],3)) + ' '
            else:
                break
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:64: UserWarning: X1 < 0 in box. Set it to 0.
  warnings.warn('X1 < 0 in box. Set it to 0.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:76: UserWarning: Y1 < 0 in box. Set it to 0.
  warnings.warn('Y1 < 0 in box. Set it to 0.')


In [52]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('not_weighted_ensemble_plus_htc_min.csv')

submission.head()

,PredictionString,image_id
0,7 0.918427 604.291 519.056 953.369 1022.949 7 ...,test/0000.jpg
1,5 0.703346 132.973 0.377 494.719 280.018 6 0.5...,test/0001.jpg
2,1 0.744817 424.662 268.814 642.111 326.28 4 0....,test/0002.jpg
3,9 0.533196 146.353 256.802 920.591 822.479 9 0...,test/0003.jpg
4,1 0.554408 210.347 264.402 868.862 772.972 0 0...,test/0004.jpg


## nms + soft_nms + wbf + nmw

In [12]:
# ensemble csv files
submission_files = ['swinB_cascade_swinB_HTC_nmw.csv',
                    'swinB_cascade_swinB_HTC_nms.csv',
                    'swinB_cascade_swinB_HTC_soft_nms.csv',
                    'swinB_cascade_swinB_HTC_wbf.csv'
                    ] # submission lists
submission_df = [pd.read_csv(file) for file in submission_files]

image_ids = submission_df[0]['image_id'].tolist()

annotation = '/opt/ml/detection/dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [13]:
prediction_strings = []
file_names = []
iou_thr = 0.4

ensemble_function =  soft_nms

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = ensemble_function(boxes_list, scores_list, labels_list, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 3 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Che

Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 2 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]
Warning. Fixed 1 boxes coordinates > 1. Check that your boxes was normalized at [0, 1]


In [14]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('swinB_cascade_swinB_HTC_total_soft_nms.csv')

submission.head()

,PredictionString,image_id
0,0 0.30352563 489.4098815917969 585.01861572265...,test/0000.jpg
1,0 0.17676884 759.9013671875 663.2786865234375 ...,test/0001.jpg
2,0 0.9725914 880.3657836914062 462.399993896484...,test/0002.jpg
3,0 0.048839312 6.2980804443359375 0.34271004796...,test/0003.jpg
4,0 0.93200076 424.5522766113281 406.36660766601...,test/0004.jpg
